In [6]:
!pip install openvino
!pip install onnx
# the torch require onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 52.8 MB/s eta 0:00:00


In [7]:
import datetime
import time


In [23]:

import numpy as np
import torch.nn.functional as F
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn, optim
from openvino.runtime import Core


class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # Convolutional Block 1
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Convolutional Block 2
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Convolutional Block 3
        self.conv5 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(128)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn6 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)

        # Fully Connected Layers
        self.fc1 = nn.Linear(128 * 4 * 4, 1024)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(1024, 512)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(512, 10)  # Output layer: 10 classes for MNIST

    def forward(self, x):
        # Convolutional Block 1
        x = self.pool1(F.relu(self.bn2(self.conv2(F.relu(self.bn1(self.conv1(x)))))))
        # Convolutional Block 2
        x = self.pool2(F.relu(self.bn4(self.conv4(F.relu(self.bn3(self.conv3(x)))))))
        # Convolutional Block 3
        x = self.pool3(F.relu(self.bn6(self.conv6(F.relu(self.bn5(self.conv5(x)))))))
        # Flatten the output for the fully connected layer
        x = x.view(-1, 128 * 4 * 4)
        # Fully connected layers with dropout in between
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x



transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Original model training
for epoch in range(1):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}')

print('Finished Training')


# Original model testing
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)


def test_model(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy


accuracy = test_model(model, testloader)
print(f'Accuracy of the model on the 10000 test images: {accuracy} %')









Epoch 1, Loss: 0.45273880164490454
Finished Training
Accuracy of the model on the 10000 test images: 98.41 %
Model has been converted to ONNX format and saved as simple_model.onnx


In [31]:
###### The following is openvino part


dummy_input = torch.randn(64, 1, 28, 28)



torch.onnx.export(model, dummy_input, "complex_gpu_model.onnx", export_params=True,
                          opset_version=11, do_constant_folding=True,
                          input_names=['input'], output_names=['output'],
                          dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}})
print("Model has been converted to ONNX format and saved as simple_model.onnx")




ie = Core()

model_path = "simple_model.onnx"
ovmodel = ie.read_model(model=model_path)

# Compile the model for a specific device
cuta = time.time()
compiled_ovmodel = ie.compile_model(model=ovmodel, device_name="CPU")
cutb = time.time()
print("compile time:",cutb-cuta)
#input_layer = compiled_ovmodel.input(0)
#output_layer = compiled_ovmodel.output(0)



Model has been converted to ONNX format and saved as simple_model.onnx
compile time: 0.025894880294799805


In [29]:
for i in range(5):
    model.eval()
    correct = 0
    total = 0
    ori_t_s=0
    for data in testloader:
        images, labels = data
        ori_t_a=time.time()
        outputs = model(images)
        ori_t_b=time.time()
        ori_t_s= ori_t_s + ori_t_b - ori_t_a

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the 10000 test images: {accuracy} %')
    print("exact time for original model: ",ori_t_s)



    correct = 0
    total = 0
    vino_t_s=0
    for data in testloader:
        images, labels = data
        vino_t_a=time.time()
        outputs = compiled_ovmodel(images)[compiled_ovmodel.output(0)]
        vino_t_b=time.time()
        vino_t_s= vino_t_s + vino_t_b - vino_t_a
        outputs = torch.from_numpy(outputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy of the vinomodel on the 10000 test images: {accuracy} %')
    print("exact time for vino model: ",vino_t_s)

Accuracy of the model on the 10000 test images: 98.41 %
exact time for original model:  14.936014175415039
Accuracy of the vinomodel on the 10000 test images: 98.41 %
exact time for vino model:  11.024291515350342
Accuracy of the model on the 10000 test images: 98.41 %
exact time for original model:  16.69730544090271
Accuracy of the vinomodel on the 10000 test images: 98.41 %
exact time for vino model:  10.255162715911865
Accuracy of the model on the 10000 test images: 98.41 %
exact time for original model:  15.086392164230347
Accuracy of the vinomodel on the 10000 test images: 98.41 %
exact time for vino model:  10.273751258850098
Accuracy of the model on the 10000 test images: 98.41 %
exact time for original model:  14.833641290664673
Accuracy of the vinomodel on the 10000 test images: 98.41 %
exact time for vino model:  10.300639152526855
Accuracy of the model on the 10000 test images: 98.41 %
exact time for original model:  14.824760437011719
Accuracy of the vinomodel on the 10000